In [205]:
import pandas as pd # Data preperation and data readings 
import numpy as np 
import matplotlib.pyplot as plt # Plots
import seaborn as sb # Prettier plots
import folium # Maps
from branca.element import Figure

In [215]:
weather_ds = pd.read_csv("weather-data-2014-2019.csv",header=None)
weather_ds_copy = weather_ds.copy()
weather_ds_copy.drop(weather_ds.columns[[2,3,4,6,7,8,9,10,12,14,16,18,20,22,23,24,25,26,27,28,29,30]],axis=1,inplace=True)
weather_ds_copy.head()

C:\Users\eskil\Anaconda3\envs\california\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,5,11,13,15,17,19,21
0,2,1/1/2014,0,64.7,29.4,44.2,74,30,53
1,2,1/2/2014,0,63.9,28.2,44.3,68,31,51
2,2,1/3/2014,0,64.1,30,45,75,31,53
3,2,1/4/2014,0,63.5,31.3,44.6,74,32,54
4,2,1/5/2014,0,64.4,29.1,44.4,72,34,57


In [217]:

weather_ds_copy.columns = ['Station Number','Date', 'Precipitation','Maximum Air Temperature','Minimum Air Temperature','Average Air Temperature','Maximum Relative Humidity','Minimum Relative Humidity','Average Relative Humidity']
weather_ds_copy.head()

0           2
1           2
2           2
3           2
4           2
         ... 
324273    261
324274    261
324275    261
324276    261
324277      
Name: Station Number, Length: 324278, dtype: object


In [208]:

for column in ['Minimum Air Temperature','Maximum Air Temperature','Average Air Temperature']:
    new_temperature = []
    for v in weather_ds_copy[column]:
        if v == '--':
            new_temperature.append(v)
            continue
        celsius = round((float(v)-32) * (5/9),1)
        new_temperature.append(celsius)
    weather_ds_copy[column] = new_temperature
weather_ds_copy.head()


,Date,Precipitation,Maximum Air Temperature,Minimum Air Temperature,Average Air Temperature,Maximum Relative Humidity,Minimum Relative Humidity,Average Relative Humidity
0,1/1/2014,0,18.2,-1.4,6.8,74,30,53
1,1/2/2014,0,17.7,-2.1,6.8,68,31,51
2,1/3/2014,0,17.8,-1.1,7.2,75,31,53
3,1/4/2014,0,17.5,-0.4,7.0,74,32,54
4,1/5/2014,0,18.0,-1.6,6.9,72,34,57


In [213]:
station_ds = pd.read_excel('CIMIS Stations List (January20).xlsx')
station_ds.head()

,Station Number,DWR Regional Office*,Name,County,Latitude,Longitude,ELEV,Status,Connect,Disconnect
0,1.0,SCRO,Fresno/F.S.U. USDA,Fresno,36.814000,-119.732000,340.0,Inactive,1982-06-07,1988-09-25 00:00:00
1,2.0,SCRO,Five Points,Fresno,36.336222,-120.112906,285.0,Active,1982-06-07,Active
2,3.0,SCRO,Beach /Santa Cruz CO,Santa Cruz,36.881000,-121.793000,10.0,Inactive,1982-05-30,1986-08-25 00:00:00
3,4.0,SCRO,Webb /Santa Cruz CO,Santa Cruz,36.972000,-121.726000,230.0,Inactive,1982-05-30,1988-04-29 00:00:00
4,5.0,SCRO,Shafter/USDA,Kern,35.532695,-119.281862,360.0,Active,1982-06-01,Active


In [221]:
station_ds_copy = station_ds.copy()
station_ds_copy['Station Number'].isin(weather_ds_copy['Station Number'])


OPPGAVER: 

1. wildfires per month
2. wildfires avg acresburned per year

In [209]:
ds = pd.read_csv("California_Fire_Incidents.csv")
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1636 entries, 0 to 1635
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AcresBurned           1633 non-null   float64
 1   Active                1636 non-null   bool   
 2   AdminUnit             1636 non-null   object 
 3   AirTankers            28 non-null     float64
 4   ArchiveYear           1636 non-null   int64  
 5   CalFireIncident       1636 non-null   bool   
 6   CanonicalUrl          1636 non-null   object 
 7   ConditionStatement    284 non-null    object 
 8   ControlStatement      111 non-null    object 
 9   Counties              1636 non-null   object 
 10  CountyIds             1636 non-null   object 
 11  CrewsInvolved         171 non-null    float64
 12  Dozers                123 non-null    float64
 13  Engines               191 non-null    float64
 14  Extinguished          1577 non-null   object 
 15  Fatalities           

In [210]:
firedep_ds = pd.read_excel("CalFireDepartments.xlsx")

firedep_ds.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
cols=["Location", "SearchDescription", "SearchKeywords", 
"CanonicalUrl", "Final", "Featured", "FuelType", "Updated", 
"StructuresEvacuated", "Active", "ConditionStatement", 
"ControlStatement", "Public", "PercentContained", "Status"]
ds_copy = ds.copy()

ds_copy = ds_copy.drop(cols, axis = 1)
ds_copy["Extinguished"] = pd.to_datetime(ds_copy["Extinguished"])
ds_copy["Started"] = pd.to_datetime(ds_copy["Started"])
ds_copy.info()

In [ ]:
# Segmentating data and adding aditional information.

# Add the time of the fire duration.
ds_copy["FireDuration"] = -(ds_copy["Started"]-ds_copy["Extinguished"])

#Segmentating the data based on how big (how many acres burned) the wild fire is.
meanAcresBurned = ds_copy["AcresBurned"].mean()
ds_copy.loc[ds_copy["AcresBurned"]>meanAcresBurned+10000, "WildfireSize"] = "BigWildfire"
ds_copy.loc[ds_copy["AcresBurned"]<=meanAcresBurned+10000, "WildfireSize"] = "AvgWildfire"
ds_copy.loc[ds_copy["AcresBurned"]<=meanAcresBurned-2000, "WildfireSize"] = "SmallWildfire"
ds_copy.head()

In [ ]:
# Add additional column with bool value where the av fire duration is lower than the mean of dataset without outliers.
lowerOutlier = ds_copy["FireDuration"].quantile(0.05) # Remove lower outliers
upperOutlier = ds_copy["FireDuration"].quantile(0.95) # Remove upper outliers
ds_copy1 = ds_copy[(ds_copy["FireDuration"]>lowerOutlier) & (ds_copy["FireDuration"]<upperOutlier)] # New dataset for mean without outliers
meanFireDuration = ds_copy1["FireDuration"].mean() # Mean of new dataset

ds_copy.loc[ds_copy["FireDuration"]>meanFireDuration, "Fast Extinguishing"] = False
ds_copy.loc[ds_copy["FireDuration"]<=meanFireDuration, "Fast Extinguishing"] = True

# map values 0 to false and 1 to true
ds_copy["Fast Extinguishing"] = ds_copy["Fast Extinguishing"].astype("bool")
ds_copy.info()

In [ ]:
# Drop rows where the longitude and latitude is outside California
# California Latitude: 32 to 42
# California Longitude: -114 to -124
ds_copy = ds_copy[ds_copy.Latitude>32]
ds_copy = ds_copy[ds_copy.Latitude<42]
ds_copy = ds_copy[ds_copy.Longitude<-114]
ds_copy = ds_copy[ds_copy.Longitude>-124]

In [ ]:
# Number of wildfires per year
plt.title("Number of Wildfire Cases per Year", fontdict=dict(weight='bold'))
numWildfires = sb.countplot(x="ArchiveYear", data=ds_copy, palette="autumn")
numWildfires.set_xlabel(xlabel='Year', fontdict=dict(weight='bold'))
numWildfires.set_ylabel(ylabel='Amount of Wildfires', fontdict=dict(weight='bold'))

In [ ]:
plt.title("Acres Burned per Year", fontdict=dict(weight="bold"))
acresBurnedPerYear = sb.barplot(x="ArchiveYear", y="AcresBurned", data=ds_copy, ci=0, palette="autumn")
acresBurnedPerYear.set_xlabel(xlabel= "Year", fontdict=dict(weight='bold'))
acresBurnedPerYear.set_ylabel(ylabel='Acres Burned', fontdict=dict(weight='bold'))

In [ ]:
# Correlation between columns and "Fast Extinguishing"
corr_ds = ds_copy.drop(["UniqueId", "Counties", "CountyIds", "Extinguished", "Started", "Name", "FireDuration", "AdminUnit", "WildfireSize", "ArchiveYear"], axis=1)
x = round(corr_ds.corr(),2)
plt.figure(figsize=(18,14))
sb.heatmap(x[["Fast Extinguishing"]].sort_values(by=["Fast Extinguishing"], ascending=False), cmap='coolwarm', annot=True)
plt.show()

In [ ]:
# Pair plot columns from the dataset and show for which wildfire size they are from
pairplot = sb.PairGrid(data=ds_copy, vars = ["AcresBurned", "ArchiveYear", "CrewsInvolved"], hue="Fast Extinguishing", height=3)
pairplot.map_diag(plt.hist)
pairplot.map_offdiag(plt.scatter)
pairplot.add_legend()

In [ ]:
# Create a folium map and place a circle in the map for each of the wildfires from 2013-2020 with a larger radius depending on how big the wildfire was 
# ; uncomment the layercontrol line if you want other maptypes
fig=Figure(width=550,height=350)

map = folium.Map(location=[36.778259, -119.417931], zoom_start=5)
fig.add_child(map)
folium.TileLayer('Stamen Terrain').add_to(map)
#folium.LayerControl().add_to(map)
for i in range (0, ds_copy.shape[0]):
  r = np.sqrt(ds_copy.iloc[i]["AcresBurned"])*20
  folium.Circle(location=[ds_copy.iloc[i]["Latitude"], ds_copy.iloc[i]["Longitude"]], radius=r, color = "red", fill_color = "orange").add_to(map)
map

In [ ]:
# Create a folium map for each of the wildfires under the category of "BigWildfire"; uncomment the layercontrol line if you want other maptypes
from folium.plugins import MarkerCluster
fig2=Figure(width=550,height=350)

bigFireMap = folium.Map(location=[36.778259, -119.417931], zoom_start=5)
cluster_marker = MarkerCluster().add_to(bigFireMap)
fig2.add_child(bigFireMap)
folium.TileLayer('Stamen Terrain').add_to(bigFireMap)
#folium.LayerControl().add_to(bigFireMap)
for i in range (0, ds_copy.shape[0]):
  r = np.sqrt(ds_copy.iloc[i]["AcresBurned"])/20
  if ds_copy.iloc[i]["WildfireSize"] == "BigWildfire":
    folium.CircleMarker(location=[ds_copy.iloc[i]["Latitude"], ds_copy.iloc[i]["Longitude"]], radius=r, color = "red", fill_color = "orange").add_to(cluster_marker)
bigFireMap

In [ ]:
# Heat maps with visualization of County CAL units
from folium import plugins
from folium.plugins import HeatMap
fig3=Figure(width=1200,height=900)

heat_map = folium.Map(location=[36,-120], zoom_start=5)

fig3.add_child(heat_map)
folium.TileLayer('Stamen Terrain').add_to(heat_map)

HeatMap(data=ds_copy[['Latitude', 'Longitude']], radius=15).add_to(folium.FeatureGroup(name='Heat Map').add_to(heat_map))
folium.LayerControl().add_to(heat_map)
for i in range (0, firedep_ds.shape[0]):
    #folium.Circle(location=[firedep_ds.iloc[i]["Latitude"], firedep_ds.iloc[i]["Longitude"]], radius=500, color = "black", fill=True, fill_opacity= 1, tooltip=firedep_ds.iloc[i]["Department Name"]).add_to(heat_map)
    folium.map.Marker(location=[firedep_ds.iloc[i]["Latitude"], firedep_ds.iloc[i]["Longitude"]], icon=folium.Icon(color="black", icon="fire-extinguisher", prefix="fa"), tooltip=firedep_ds.iloc[i]["Department Name"]).add_to(heat_map)
heat_map # display

In [ ]:
# Create a folium map and place a circle in the map for each of the wildfires from 2013-2020 with a larger radius depending on how big the wildfire was 
# ; uncomment the layercontrol line if you want other maptypes
fig=Figure(width=550,height=350)

hot_spot = folium.Map(location=[36.778259, -119.417931], zoom_start=5)
cluster_marker = MarkerCluster().add_to(hot_spot)
fig.add_child(hot_spot)
folium.TileLayer('Stamen Terrain').add_to(hot_spot)
#folium.LayerControl().add_to(map)
for i in range (0, ds_copy.shape[0]):
  r = np.sqrt(ds_copy.iloc[i]["AcresBurned"])/50
  folium.CircleMarker(location=[ds_copy.iloc[i]["Latitude"], ds_copy.iloc[i]["Longitude"]], radius=r, color = "red", fill_color = "orange").add_to(cluster_marker)

for i in range (0, firedep_ds.shape[0]):
    #folium.Circle(location=[firedep_ds.iloc[i]["Latitude"], firedep_ds.iloc[i]["Longitude"]], radius=500, color = "black", fill=True, fill_opacity= 1, tooltip=firedep_ds.iloc[i]["Department Name"]).add_to(heat_map)
    folium.map.Marker(location=[firedep_ds.iloc[i]["Latitude"], firedep_ds.iloc[i]["Longitude"]], icon=folium.Icon(color="black", icon="fire-extinguisher", prefix="fa"), tooltip=firedep_ds.iloc[i]["Department Name"]).add_to(hot_spot)
hot_spot

In [ ]:
# Heat maps with visualization per year
from folium import plugins
from folium.plugins import HeatMap
fig3=Figure(width=550,height=350)

heat_map2 = folium.Map(location=[36,-120], zoom_start=5)

fig3.add_child(heat_map2)
folium.TileLayer('Stamen Terrain').add_to(heat_map2)

for val in ds_copy["ArchiveYear"].unique():
    df = ds_copy[ds_copy.ArchiveYear == val]
    HeatMap(data=df[['Latitude', 'Longitude']], radius=20).add_to(folium.FeatureGroup(name=str(val), show=False).add_to(heat_map2))


folium.LayerControl().add_to(heat_map2)

heat_map2

In [ ]:
# Clustering of wildfires per year
from folium import plugins
fig=Figure(width=550,height=350)

hot_spot2 = folium.Map(location=[36.778259, -119.417931], zoom_start=5)
fig.add_child(hot_spot2)
folium.TileLayer('Stamen Terrain').add_to(hot_spot2)

for val in ds_copy["ArchiveYear"].unique():
  df = ds_copy[ds_copy.ArchiveYear == val]
  cluster_marker2 = MarkerCluster(name=str(val), show=False).add_to(hot_spot2)
  for i in range (0, df.shape[0]):
    r = np.sqrt(df.iloc[i]["AcresBurned"])/50
    folium.CircleMarker(location=[df.iloc[i]["Latitude"], df.iloc[i]["Longitude"]], radius=r, color = "red", fill_color = "orange").add_to(cluster_marker2)

folium.LayerControl().add_to(hot_spot2)

hot_spot2

In [ ]:
ds_copy["ArchiveMonth"] = pd.DatetimeIndex(ds_copy["Started"]).month.astype(str) + pd.DatetimeIndex(ds_copy["Started"]).year.astype(str)
plt.title("Number of wildfires per month", fontdict=dict(fontweight="bold"))
plt.rcParams["figure.figsize"] = (15,6)
numWildfiresPerMonth = sb.countplot(x="ArchiveMonth", data=ds_copy, palette="autumn")
numWildfiresPerMonth.set_xlabel(xlabel="Year", fontdict=dict(fontweight="bold"))
numWildfiresPerMonth.set_ylabel(ylabel="Number of wildfires", fontdict=dict(fontweight="bold"))
numWildfiresPerMonth.set_xticklabels(numWildfiresPerMonth.get_xticklabels(), rotation=90, ha="right")